In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.9 MB/s eta 0:00:00


In [ ]:
!pip3 install wikipedia-api

In [ ]:
!pip install transformers
!pip install evaluate
!pip install transformers[torch]
!pip install accelerate -U
!pip install sentencepiece
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import string
import nltk
import contractions
import wikipediaapi
from transformers import AutoModelForSeq2SeqLM,T5ForConditionalGeneration,T5Tokenizer, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import accelerate
import bitsandbytes
import pandas as pd
import numpy as np
import re
import os
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##**Data Collection**

**Get data from Wikipedia**

In [ ]:
def wiki_data(topic="programming language", language='en'):

  wiki_wiki = wikipediaapi.Wikipedia(
          user_agent='Api-User-Agent',
          language= language,
          extract_format=wikipediaapi.ExtractFormat.WIKI
  )

  p_wiki = wiki_wiki.page(topic)
  data = p_wiki.text.splitlines()

  return data

##**TEXT-PREPROCESSING**

In [ ]:
def remove_pg_num(text):
    text = re.sub('[[]\d+[]]|[[]\d[]]', '', text)
    return text


In [ ]:
def remove_extra_spaces(text):
  text = re.sub('\s+', ' ', text)
  return text

In [ ]:
def remove_url(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

In [ ]:
def remove_ar(text):
    text = re.sub('[أ-ي]', '', text)
    return text

In [ ]:
def remove_num(text):
    text = re.sub('[0-9]', '', text)
    return text

In [ ]:
def remove_mails(text):
    text = re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*', "", text)
    return text

In [ ]:
def remove_dot(text):
    text = re.sub(r'\.+', ".", text)
    text = re.sub(r'\,+', ",", text)
    return text

In [ ]:
def remove_punctuations(text):
    punctuations = '!"#$%&\()*/:;<=>?@[\\]^_`{|}~-'
    return text.translate(str.maketrans('', '', punctuations))

In [ ]:
def remove_single_quotes(text):
    text = re.sub("'", '', text)
    return text

**TEXT-PREPROCESSING (FUNCTION)**

In [ ]:
# The parameter list is list of text-lines
def text_preprocessing(list):
  #if there is any (\n) in the end of each line
  for line in list:
    line = line.strip()
  #To concatenate all text-data
  data = " ".join([line for line in list])

  data = remove_pg_num(data)
  data = remove_url(data)
  data = remove_ar(data)
  data = remove_mails(data)
  data = contractions.fix(data)
  data = remove_punctuations(data)
  data = remove_num(data)
  data = remove_dot(data)
  data = remove_single_quotes(data)
  data = remove_extra_spaces(data)

  return data





##Question Generation

In [ ]:
!pip install lmqg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.6/662.6 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00
  Created wheel for lmqg: filename=

In [ ]:
from lmqg import TransformersQG

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
def model_loading(path):

  loaded_model = TransformersQG(language="en", model="lmqg/flan-t5-base-squad-qg")

  return loaded_model

In [ ]:
loaded_model = model_loading('/content/drive/MyDrive/Models/Qg_Model')


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1006: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2356: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 32101. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [ ]:
def generate_q_(context, model= loaded_model):

    question_output = loaded_model.generate_q(list_context=context)

    return question_output

In [ ]:
def generate_q(context, model= loaded_model, tokenizer= loaded_tokenizer):

    prompt_template = f"""
    Generate Question from the following context.

    {context}

    question:
    """
    input_ids = tokenizer(prompt_template, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda') if model.device.type == 'cuda' else input_ids

    model_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=512, num_beams=2))
    question_output = tokenizer.decode(model_outputs[0], skip_special_tokens=True)

    return question_output

##Answer Generation

In [ ]:
from transformers import BartForConditionalGeneration
from transformers import pipeline

def AQ_load():
  question_answerer = pipeline("question-answering", model="/content/drive/MyDrive/Models/AG_Model")
  return question_answerer

AQ_MODEL = AQ_load()

In [ ]:
def AQ_predict(context, question):
    return AQ_MODEL(question=question, context=context)['answer']

##Distractors Generator

In [ ]:
#import
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import random

#GPU -> this for hole project
if torch.cuda.is_available():
    torch_device = 'cuda'
else:
    torch_device = 'cpu'
torch_device

'cpu'

In [ ]:
#VIP model folder https://drive.google.com/drive/folders/1Y7B3EBI7TcK7VvppdFrUJ97BQwxhKHx_?usp=sharing choose 30000
def load_Distractor_model(Distractor_path = "", distractor_model_type = "t5-small", max_length = 512):
  distractor_model = AutoModelForSeq2SeqLM.from_pretrained('potsawee/t5-large-generation-race-Distractor')
  if torch_device == "cuda":
      distractor_model.cuda()
      state = torch.load(Distractor_path) # path to Distractor Gen (t5-large)
  else:
      state = torch.load(Distractor_path, map_location=torch.device('cpu'))

  # model_state_dict = state['model']
  # distractor_model.load_state_dict(model_state_dict)
  # distractor_model.eval()
  print('DistractorGeneration Model loaded:', Distractor_path)

  distractor_tokenizer = AutoTokenizer.from_pretrained('potsawee/t5-large-generation-race-Distractor', model_max_length=max_length)
  distractor_tokenizer.add_special_tokens({"sep_token": "<sep>"})

  return distractor_model, distractor_tokenizer

distractor_model, distractor_tokenizer = load_Distractor_model(Distractor_path = "/content/drive/MyDrive/Models/t5-small-Race-Distractor-Generation-version0-step0.pt", distractor_model_type = "t5-small", max_length = 512)
#VIP distractor_tokenizer --> path to

DistractorGeneration Model loaded: /content/drive/MyDrive/Models/t5-small-Race-Distractor-Generation-version0-step0.pt


In [ ]:
context = r"""
World number one Novak Djokovic says he is hoping for a "positive decision" to allow him
to play at Indian Wells and the Miami Open next month. The United States has extended
its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination
will be required to enter the country until at least 10 April, but the Serbian has previously
said he is unvaccinated. The 35-year-old has applied for special permission to enter the country.
Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar
outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to
the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title
and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")

question = "What is the best title for the passage?"
answer = "Djokovic's application for special permission to enter the United States"


In [ ]:
def generate_distractors(t5_tokenizer, context = "",question = "" ,answer = "" ,separator='<sep>', max_length=512, torch_device='cpu',):
  #output ==> dis <sep> dis <> dis
  input_text = question + ' ' + separator + ' ' + answer + ' ' + separator + ' ' + context
  encoding = t5_tokenizer(
      [input_text],
      padding="longest",
      max_length=max_length,
      truncation=True,
      return_tensors="pt",)

  input_ids = encoding.input_ids

  if torch_device == 'cuda':
      input_ids = input_ids.cuda()


  outputs = distractor_model.generate(
  input_ids,
  max_new_tokens=256,
  do_sample=True,)

  distractors = distractor_tokenizer.decode(outputs[0], skip_special_tokens=False)
  distractors = distractors.replace(distractor_tokenizer.pad_token, "").replace(distractor_tokenizer.eos_token, "")
  distractors = [y.strip() for y in distractors.split(distractor_tokenizer.sep_token)]
  #[gg,gg,gg]
  options = [answer] + distractors

  if "" or '' in options:
    try:
      options.remove("")
    except:
      options.remove('')

  random.shuffle(options)
  return options

prefix = ['A', 'B' , 'C', 'D']
options = generate_distractors(distractor_tokenizer, context = context,question = question ,answer = answer ,separator='<sep>', max_length=512, torch_device= torch_device,)
print(f"Q: {question} \n ")
for i , option in zip(prefix, options):
  print(f"  {i}: {option} \n")
print(f"correct: {answer}")



Q: What is the best title for the passage? 
 
  A: Djokovic's expectations about the Miami Open 

  B: Djokovic's invitation to participate in the Miami Open 

  C: Djokovic's application for special permission to enter the United States 

  D: Djokovic's plans to come back to the ATP tour 

correct: Djokovic's application for special permission to enter the United States


##ChatBot

In [ ]:
import torch
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from transformers.generation.utils import GreedySearchDecoderOnlyOutput

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cpu'

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LlamaForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Models/LLaMA-lora-alpaca",
    load_in_8bit=True,
    device_map="auto",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


ImportError: ignored

In [ ]:
!pip install bitsandbytes versio

In [ ]:
PROMPT_TEMPLATE = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
[INSTRUCTION]

### Response:
"""

In [ ]:
def create_prompt(instruction: str) -> str:
    return PROMPT_TEMPLATE.replace("[INSTRUCTION]", instruction)

print(create_prompt("What is the meaning of life?"))


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is the meaning of life?

### Response:



In [ ]:
def generate_response(prompt: str, model: PeftModel, max_new_tokens) -> GreedySearchDecoderOnlyOutput:
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to(DEVICE)

    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        repetition_penalty=1.1,
    )
    with torch.inference_mode():
        return model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )

In [ ]:
def format_response(response: GreedySearchDecoderOnlyOutput) -> str:
    decoded_output = tokenizer.decode(response.sequences[0])
    response = decoded_output.split("### Response:")[1].strip()
    return "\n".join(textwrap.wrap(response))

In [ ]:
def ask_alpaca(prompt: str, max_new_tokens, model: PeftModel = model) -> str:
    prompt = create_prompt(prompt)
    response = generate_response(prompt, model, max_new_tokens)
    return format_response(response)

In [ ]:
out = ask_alpaca("Generate 2 MCQ and answers about ML", 100)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
out

'1) What are the different types of Machine Learning? - Supervised\nlearning, Unsupervised learning, Reinforcement learning 2) What are\nthe different algorithms used in Machine Learning? - Decision tree,\nNeural Networks, Random Forest, KNN, SVM, Naive Bayes, Logistic\nRegression, Gradient Boosting, Ridge Regression, Linear Regression,\nSupport Vector Machines, K-Means Clustering, Principal Component'

In [ ]:
out.split('### Instruction:')[0]

'1) What are the different types of Machine Learning? - Supervised\nlearning, Unsupervised learning, Reinforcement learning 2) What are\nthe different algorithms used in Machine Learning? - Decision tree,\nNeural Networks, Random Forest, KNN, SVM, Naive Bayes, Logistic\nRegression, Gradient Boosting, Ridge Regression, Linear Regression,\nSupport Vector Machines, K-Means Clustering, Principal Component'

##Demo

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.4 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579 

In [ ]:
import gradio as gr
from google.colab import files

def generate_question_(context):
    text_preprocessed = text_preprocessing([context])
    q = []
    a = []
    c = []
    A = []
    B = []
    C = []
    D = []
    for sentence in text_preprocessed.split('.'):
        if len(sentence.split()) > 10:
            q.append(generate_q(sentence))
            a.append(AQ_predict(sentence,q[-1]))
            options = generate_distractors(distractor_tokenizer, context = sentence,question = q[-1] ,answer = a[-1] ,separator='<sep>', max_length=512, torch_device= torch_device,)
            A.append(options[0])
            try:
              B.append(options[1])
            except:
              B.append(A[-1])
            try:
              C.append(options[2])
            except:
              C.append(B[-1])
            try:
              D.append(options[3])
            except:
              D.append(C[-1])
            c.append(sentence)


    df = {}
    df["Context"] = c
    df["Question"] = q
    df["A"] = A
    df["B"] = B
    df["C"] = C
    df["D"] = D
    df["Correct_Answer"] = a
    df = pd.DataFrame(df)
    file_path = '/content/MCQ.csv'
    csv = df.to_csv(file_path)
    return file_path

def generate_question_wiki(topic):
    data = wiki_data(topic=topic)
    text_preprocessed = text_preprocessing(data)
    q = []
    a = []
    c = []
    A = []
    B = []
    C = []
    D = []
    for sentence in text_preprocessed.split('.'):
        if len(sentence.split()) > 10:
            q.append(generate_q(sentence, loaded_model, loaded_tokenizer))
            a.append(AQ_predict(sentence,q[-1]))
            options = generate_distractors(distractor_tokenizer, context = context,question = question ,answer = answer ,separator='<sep>', max_length=512, torch_device= torch_device,)
            A.append(options[0])
            try:
              B.append(options[1])
            except:
              B.append(A[-1])
            try:
              C.append(options[2])
            except:
              C.append(B[-1])
            try:
              D.append(options[3])
            except:
              D.append(C[-1])
            c.append(sentence)


    df = {}
    df["Context"] = c
    df["Question"] = q
    df["A"] = A
    df["B"] = B
    df["C"] = C
    df["D"] = D
    df["Correct_Answer"] = a
    df = pd.DataFrame(df)
    file_path = '/content/MCQ.csv'
    csv = df.to_csv(file_path)
    return file_path

def chatbot(context, number):
  txt = ask_alpaca("Generate 2 MCQ and answers about ML", 100).split('### Instruction:')[0]
  text_file = open("Output.txt", "w")

  text_file.write(txt)

  text_file.close()
  return "Output.txt"

with gr.Blocks() as demo:
    gr.Markdown('MCQ Generator')
    with gr.Tab("Paragraph"):
        text_input = gr.Textbox(placeholder='Insert Paragraph...')
        file_ = gr.components.File()
        text_button = gr.Button("Generate")
        text_button.click(generate_question_, inputs=text_input, outputs=file_)

    with gr.Tab("Wikipedia"):
        with gr.Row():
            text_input_wiki = gr.Textbox(placeholder='Insert topic like ( Machine Learning)...')
            file_wiki = gr.components.File()
            text_button_wiki = gr.Button("Generate")
            text_button_wiki.click(generate_question_wiki, inputs=text_input_wiki, outputs=file_wiki)

    with gr.Tab("Chatbot"):
        with gr.Row():
            text_input_Chatbot = gr.Textbox(placeholder='Insert command like ( Generate 5 MCQ and answers about ML )...')
            num_input_Chatbot = gr.inputs.Number()
            file_output_Chatbot = gr.components.File()
            text_button_Chatbot = gr.Button("Generate")
            text_button_Chatbot.click(chatbot, inputs=[text_input_Chatbot, num_input_Chatbot], outputs=file_output_Chatbot)

demo.queue().launch(share=True, debug=True)

<ipython-input-37-838c9f9bea6c>:118: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  num_input_Chatbot = gr.inputs.Number()
<ipython-input-37-838c9f9bea6c>:118: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  num_input_Chatbot = gr.inputs.Number()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e745ee0e4ede647fe8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


100%|██████████| 1/1 [00:00<00:00, 395.69it/s]
